# Unsupervised Valence Assignment for Sentiment Analysis

In [ ]:
import urllib
import numpy as np
import json
import re
import sys
import time
import sqlite3 as sl
import pandas as pd
from bs4 import BeautifulSoup
import scipy.sparse

from sklearn.preprocessing import normalize

from scipy.spatial import distance

In [ ]:
first_known_file = 1
last_known_file = 67790
#last_known_file = 1000
max_file_size = 0

In [ ]:
'''
for web_idx in range(first_known_file, last_known_file):
    url = f"https://gutenberg.org/files/{web_idx}/"
    try:
        file = urllib.request.urlopen(url)
    except urllib.error.HTTPError as e: # If the page is forbiden, then move on
        continue
    except urllib.error.URLError as e: # if we loose connection, try again
        time.sleep(4)
        web_idx -= 4

    # Looking for every .txt.utf-8 file given the above url
    online_file = list()
    for line in file:
        decoded_line = None
        try:
            decoded_line = line.decode("utf-8")
        except UnicodeDecodeError as e:
            continue

        online_file.append(decoded_line)



    # For each .txt.utf-8 file we need to create a new url to get to the file
    res = [string for string in online_file if '.txt">' in string]
    txt_urls = list()
    txt_file_name = None
    for line in res:
        result = re.search('<a href="(.*).txt">', line)
        if result != None:
            num_groups = len(result.groups())

            if num_groups > 0:
                txt_file_name = f"{result.group(1)}.txt"
                txt_urls.append(f"{url}{txt_file_name}")

    # for each .txt.utf-8 url file, read in the file and get all of the text
    txt_file_content = list()
    line_idx = 0
    for url_idx in range(len(txt_urls)):
        url = txt_urls[url_idx]

        if url == None or url == '':
            continue

        file = urllib.request.urlopen(url)
        line_by_line = list()
        for line in file:
            try:
                decoded_line = line.decode("utf-8")
            except UnicodeDecodeError as e:
                continue
            line_by_line.append(decoded_line)  
        txt_file_content.append(line_by_line)

        size_of_file = sys.getsizeof(line_by_line)
        if size_of_file > max_file_size:
            max_file_size = size_of_file
        print(f"{url}: {size_of_file} / {max_file_size}")
    
    # Write file to local disk
    #txt_file_content_np = np.asarray(txt_file_content, str)
    #txt_file_content_np = txt_file_content_np.flatten()
    if len(txt_file_content) >= 1 and len(txt_file_content[0]) >= 1:
        with open(f'raw/{txt_file_name}','w') as f:
            if f.tell() < 1024*1024*1024:
                #pickle.dump(txt_file_content_np, f)
                f.write(json.dumps(txt_file_content))
                #for line in txt_file_content:
                    #f.write('%s\n' % line)
'''

In [ ]:
#with open('raw/10.pkl','rb') as f:
#    x = pickle.load(f)
#    print(x[0][125])

## Example document


## ---------------


The Project Gutenberg EBook of Madame de Ferneuse, by Daniel Lesueur

This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you'll have to check the laws of the country where you are located before using this ebook.

Title: Madame de Ferneuse

Author: Daniel Lesueur

Release Date: March 21, 2016 [EBook #51515]

Language: French

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK MADAME DE FERNEUSE ***

*** END OF THIS PROJECT GUTENBERG EBOOK MADAME DE FERNEUSE ***

***** This file should be named 51515-0.txt or 51515-0.zip ***** This and all associated files of various formats will be found in: http://www.gutenberg.org/5/1/5/1/51515/


## ------------------

The Project Gutenberg EBook of The Meaning of Infancy, by John Fiske

This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org

Title: The Meaning of Infancy

Author: John Fiske

Release Date: May 15, 2004 [EBook #12359]

Language: English

Character set encoding: ASCII

*** START OF THIS PROJECT GUTENBERG EBOOK THE MEANING OF INFANCY ***

*** END OF THIS PROJECT GUTENBERG EBOOK THE MEANING OF INFANCY ***

## -------------------

The Project Gutenberg eBook of Traumerei, by Charles Beaumont

This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.

Title: Traumerei

Author: Charles Beaumont

Release Date: February 7, 2022 [eBook #67341]

Language: English

Produced by: Greg Weeks, Mary Meehan and the Online Distributed Proofreading Team at http://www.pgdp.net

*** START OF THE PROJECT GUTENBERG EBOOK TRAUMEREI ***

*** END OF THE PROJECT GUTENBERG EBOOK TRAUMEREI ***

## -------------------


### New Design
Right now, we are downloading all of the files, saving them as raw array format as a json file.

Alternative approach as relational database
We can generate the hypterlink by walking a list from 0 to last_known_file. We would need to check what files they have. Then, save only the heading information in each document, and remember where the starting index of the literature as well as the ending index.  

Format tipes:
<ul>
    <li>.txt</li>
    <li>-0.txt</li>
    <li>-8.txt</li>
    <li>-h/</li>
    <li>-0.zip</li>
    <li>-8.zip</li>
    <li>-h.zip</li>
</ul>

from there, we can read the txt file, assuming it can be decoded using utf-8. The reoccuring headings can allow us to save data entries into a relational database.
Document: (some of these fields may not be nan)
<ul>
    <li>Title:</li>
    <li>Author:</li>
    <li>URL:</li>
    <li>Release Date:</li>
    <li>Edition:</li>
    <li>Language:</li>
    <li>Produced by:</li>
    <li>Start_Of_doc_indx:</li>
    <li>End_of_doc_indx:</li>
    <li>(or, if we have the memory, the entire document, but for now assuming we are reading from a URL or File Stream)</li>
</ul>



Python and SQLite3

Medium post by Christopher Tao: https://towardsdatascience.com/do-you-know-python-has-a-built-in-database-d553989c87bd

Tutorialspoint python SQLlite: https://towardsdatascience.com/do-you-know-python-has-a-built-in-database-d553989c87bd

# SQLite database  

example code to work with sqlite3

In [ ]:
con = sl.connect('my-test.db')

with con:
    con.execute("""
        CREATE TABLE USER (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER
        );
    """)

sql = 'INSERT INTO USER (id, name, age) values(?, ?, ?)'
data = [
    (1, 'Alice', 21),
    (2, 'Bob', 22),
    (3, 'Chris', 23)
]

with con:
    con.executemany(sql, data)

In [ ]:
with con:
    data = con.execute("SELECT * FROM USER WHERE age <= 22")
    for row in data:
        print(row)

Integrate with Pandas

In [ ]:
df_skill = pd.DataFrame({
    'user_id': [1,1,2,2,3,3,3],
    'skill': ['Network Security', 'Algorithm Development', 'Network Security', 'Java', 'Python', 'Data Science', 'Machine Learning']
})

In [ ]:
df_skill.to_sql('SKILL', con)

In [ ]:
df = pd.read_sql('''
    SELECT s.user_id, u.name, u.age, s.skill 
    FROM USER u LEFT JOIN SKILL s ON u.id = s.user_id
''', con)
df

In [ ]:
df.to_sql('USER_SKILL', con)

Implementing Refactor

In [ ]:
db_con = sl.connect('gutenberg.db')

with db_con:
    db_con.execute("""
        CREATE TABLE LITERATURE (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            author TEXT,
            root_url TEXT,
            release_date TEXT,
            edition INTEGER,
            language TEXT,
            produced_by TEXT,
            translator TEXT,
            url_txt_0 TEXT,
            url_txt_8 TEXT,
            url_zip_0 TEXT,
            url_zip_8 TEXT,
            url_zip_h TEXT,
            url_h TEXT,
            url_txt TEXT
        );
    """)

In [ ]:
#sql = 'INSERT INTO USER (id, name, age) values(?, ?, ?)'
#data = [
#    (1, 'Alice', 21),
#    (2, 'Bob', 22),
#    (3, 'Chris', 23)
#]
#
#with con:
#    con.executemany(sql, data)

In [ ]:
#db_con = sl.connect('gutenberg.db')
#
#with con:
#    con.execute("""DROP table LITERATURE;""")

In [ ]:
TXT_0_FILE = '-0.txt'
TXT_8_FILE = '-8.txt'
ZIP_0_FILE = '-0.zip'
ZIP_8_FILE = '-8.zip'
ZIP_H_FILE = '-h.zip'
H_FILE = '-h/'
TXT_FILE = '.txt'

In [ ]:
for web_idx in range(1, last_known_file):
    # variables to populate in database
    id = web_idx
    title = "None"
    author = "None"
    root_url = f"https://gutenberg.org/files/{web_idx}/"
    release_date = "None"
    edition = 0
    language = "None"
    produced_by = "None"
    translator = "None"
    url_txt_0 = "None"
    url_txt_8 = "None"
    url_zip_0 = "None"
    url_zip_8 = "None"
    url_zip_h = "None"
    url_h = "None"
    url_txt = "None"

    # Connect to project gutenberg
    try:
        file = urllib.request.urlopen(root_url)
    except urllib.error.HTTPError as e: # If the page is forbiden, then move on
        continue
    except urllib.error.URLError as e: # if we loose connection, try again
        time.sleep(4)
        web_idx -= 4

    # Strip web string from php syntax
    online_file = list()
    for line in file:
        decoded_line = None
        try:
            decoded_line = line.decode("utf-8")
        except UnicodeDecodeError as e:
            continue
        readable_line = ''.join(BeautifulSoup(decoded_line).findAll(text=True))
        online_file.append(readable_line)
    fullstring = ''.join(online_file)

    # Parese web file for txt files  
    if f'{web_idx}{TXT_0_FILE}' in fullstring:
        url_txt_0 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{TXT_0_FILE}"
    if f'{web_idx}{TXT_8_FILE}' in fullstring:
        url_txt_8 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{TXT_8_FILE}"
    if f'{web_idx}{ZIP_0_FILE}' in fullstring:
        url_zip_0 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{ZIP_0_FILE}"
    if f'{web_idx}{ZIP_8_FILE}' in fullstring:
        url_zip_8 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{ZIP_8_FILE}"
    if f'{web_idx}{ZIP_H_FILE}' in fullstring:
        url_zip_h = f"https://gutenberg.org/files/{web_idx}/{web_idx}{ZIP_H_FILE}"
    if f'{web_idx}{H_FILE}' in fullstring:
        url_h = f"https://gutenberg.org/files/{web_idx}/{web_idx}{H_FILE}"
    if f'{web_idx}{TXT_FILE}' in fullstring:
        url_txt = f"https://gutenberg.org/files/{web_idx}/{web_idx}{TXT_FILE}"

    # use one of the txt files to gather remanining variables
    file_url = None
    if url_txt_0:
        file_url = url_txt_0
    elif url_txt_8:
        file_url = url_txt_8
    elif url_txt:
        file_url = url_txt

    if file_url == "None":
        continue

    # Connect to txt file
    file = None
    try:
        file = urllib.request.urlopen(file_url)
    except urllib.error.HTTPError as e: # If the page is forbiden, then move on
        continue
    except urllib.error.URLError as e: # if we loose connection, try again
        time.sleep(4)
        web_idx -= 4

    # Strip txt file from php syntax
    txt_file = list()
    for line in file:
        decoded_line = None
        try:
            decoded_line = line.decode("utf-8")
        except UnicodeDecodeError as e:
            continue
        #readable_line = ''.join(BeautifulSoup(decoded_line).findAll(text=True))
        txt_file.append(decoded_line)
    full_txt = ''.join(txt_file)

    #print(full_txt)

    # Parse text file for
    title_results = re.search('Title: (.*)', full_txt)
    author_results = re.search('Author: (.*)', full_txt)
    release_data_results = re.search('Release Date: (.*) \[', full_txt)
    edition_results = re.search('Edition: (.*)', full_txt)
    language_results = re.search('Language: (.*)', full_txt)
    produced_results = re.search('Produced by: (.*)', full_txt)
    translator_results = re.search('Translator: (.*)', full_txt)

    if title_results:
        title = str(title_results.group(1)).strip()
    if author_results:
        author = str(author_results.group(1)).strip()
    if release_data_results:
        release_date = str(release_data_results.group(1)).strip()
    if edition_results:
        edition_str = str(edition_results.group(1)).strip()
        if edition_str.isnumeric():
            edition = int(edition_str)
    if language_results:
        language = str(language_results.group(1)).strip()
    if produced_results:
        produced_by = str(produced_results.group(1)).strip()
    if translator_results:
        translator = str(translator_results.group(1)).strip()


    #print(f"id: {type(id)}")
    #print(f"title: {type(title)}")
    #print(f"author: {type(author)}")
    #print(f"root_url: {type(root_url)}")
    #print(f"release_date: {type(release_date)}")
    #print(f"edition: {type(edition)}")
    #print(f"language: {type(language)}")
    #print(f"produced_by: {type(produced_by)}")
    #print(f"url_txt_0: {type(url_txt_0)}")
    #print(f"url_txt_8: {type(url_txt_8)}")
    #print(f"url_zip_0: {type(url_zip_0)}")
    #print(f"url_zip_8: {type(url_zip_8)}")
    #print(f"url_zip_h: {type(url_zip_h)}")
    #print(f"url_h: {type(url_h)}")
    #print(f"url_txt: {type(url_txt)}")
    
    sql = 'INSERT INTO LITERATURE (id, title, author, root_url, release_date, edition, language, produced_by, translator, url_txt_0, url_txt_8, url_zip_0, url_zip_8, url_zip_h, url_h, url_txt) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
    data = [(id, title, author, root_url, release_date, edition, language, produced_by, translator, url_txt_0, url_txt_8, url_zip_0, url_zip_8, url_zip_h, url_h, url_txt)]

    with db_con:
        db_con.executemany(sql, data)

In [ ]:
db_con = sl.connect('gutenberg.db')

with db_con:
    db_con.execute("""DROP table LITERATURE;""")

    db_con.execute("""
        CREATE TABLE LITERATURE (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            author TEXT,
            root_url TEXT,
            release_date TEXT,
            edition INTEGER,
            language TEXT,
            produced_by TEXT,
            translator TEXT,
            url_txt_0 TEXT,
            url_txt_8 TEXT,
            url_zip_0 TEXT,
            url_zip_8 TEXT,
            url_zip_h TEXT,
            url_h TEXT,
            url_txt TEXT
        );
    """)

## Encoding Representations


- One-Hot encoding
    - Each word is represented as a single dimension

![image.png](pic/fig_1_1.png)

- Vector Space Model (VSM)
    - Refered to as the "Semantic Space"
    - Representation of the object is "Distributed Representation"
        - object can be words, documents, sentences, concepts, or entities 
        
![image.png](pic/fig_1_2.png)

Word Embeddings


Count-based models

    - term-document (row: words, col: documents) denoting the frequency of a specific word in a doc. Meant for document centirc work (document retrieval, classification, or similar document)
    - Word-context. (representing words) <---
    - pair-pattern (row: pairs of words, col: pattern occurance)



Positive Pointwise Mutual Infromation model

    - better than raw frequencies.
    - any negative values are clampped to 0
    - checks if w_1 and w_2 co-occur more than they occur independelty
    - negative values means that two words co-occurens is less than likely to happen than by chance

$ PMI(w_1, w_2) = log_2 \dfrac{P(w_1,w_2)}{P(w_1)P(w_2)}$



Cosine Similarity

    - dot product(v, w) where v and w are the embeddings of the word v and w.
    - or the cos(v,w) = (v dot w)/|v||w|


tf-idf model

    - term frequency - the frequency of a word within a document
    - inverse document frequency - the number of documents the words was found in
    - collection frequency - the total number of times a words was found in all documents
    - can look at the window as the "document"


The simplest embeddings we can do is creating a matrix that is n by n, where n is the number of unique words in the corpus. Then use the PPMI metric to calculate the probability of the words occuring at all in the corpus.


Another representation can be the word count of pairs of words, but only incrementing if the other word is +/- n words away from the target word. Essentially a sliding window, where the midpoint is the target word, and the outside words are the occurances of that word bing visable. This creates the impression of word relations between other words, with a small window allowing for specific connections, and larger windows give general connections.


Dimensionality reduction can help with the previous matrix to reduce the sparsity of the matrix. (SVD and PCA). Note that the new columns are now not naturally interpretable, whereas the oriignal embedingnig can be interpretable.


Recomended that we only keep track of the first 50,000 most frequent words, removing stop words, to reduce the sparsity of the embedding. Having more would just cause noise.


Random Indexing

    - "RI first generates unique and random lowdimensional representations for words, called index vectors. Then, in an incremental process while scanning through the text corpus, each time a word c is observed in the context of a target word w, the index vector for w is updated by adding to it the index vector of c. 
    - The resulting representations are an approximation of the co-occurrence matrix, but with a significantly lower dimensionality, and without the need for costly SVD computations. Moreover, RI representations can be update after the initial training, and once new data is at disposal."


Predictive Models

    - Using neural networks to predict the embedding space of the words.
    - Word2Vec
    - GloVe

Character Embeddings

    - out of vocaulary (OOV)
    - solution to randomly assign a value, but can hinder our understanding/accuracy
    - using morphological segmenter (memoryless --> memory and less)
    - broken into constituent subwords (groups of characters that are not neccessarily semantically meaningful)
    - FastText
    - Byte pair encoding, and WordPiece 
    - WordNet

Knowledge-enhanced word embeddings
    - retrofitting models


Evaluation of word embeddings
    - Intrinsic (generic evaluation)
    - Extrinsic (specific application use evaluation)




In [ ]:
# Need an abstract class that holds the "embeddings" method
# All we need to know is that the model works with an arbitrary embedding
# which at the end of the day it is an n by n matrix.

# This may be just a wrapper class for sklearn feature extraction 

# Sparse matrix representation by having 3 parallel lists
# row holds the row index
# col holds the col index
# data holds the value at (row, col)
class Embedding():
    def __init__(self):
        self.row = list()
        self.col = list()
        self.data = list()



# we will be working with sparse matrixies, and thus we can use scipy's sparce matrix package
# example of using their scipy.sparse.csr_array (Compressed Sparse Row array)
# row = np.array([0, 0, 1, 2, 2, 2])
# col = np.array([0, 2, 2, 0, 1, 2])
# data = np.array([1, 2, 3, 4, 5, 6])
# csr_array((data, (row, col)), shape=(3, 3)).toarray()
#    array([[1, 0, 2],
#           [0, 0, 3],
#           [4, 5, 6]])
# 
# The tuple to represent row and col are seperated in the row and col parallel vectors.
# Then a third parallel vector represents the data that we are interested in
# 
# Addtionlay, if there is a duplicate row/col entries, these are summed together
# row = np.array([0, 1, 2, 0])
# col = np.array([0, 1, 1, 0])
# data = np.array([1, 2, 4, 8])
# csr_array((data, (row, col)), shape=(3, 3)).toarray()
#   array([[9, 0, 0],
#          [0, 2, 0],
#          [0, 4, 0]])

Understanding how to use sklearn feature extractions

In [ ]:
def unique(list1):
    unique_word_list = []
    for x in list1:
        sentence_list = x.split()
        for y in sentence_list:
            lower = y.lower()
            if lower not in unique_word_list:
                unique_word_list.append(lower)
    return unique_word_list



corpus = [
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'Is this the first document',
]

vocab_list = unique(corpus)
vocab_dict = {word: idx for idx, word in enumerate(vocab_list)}
vocab_dict

In [ ]:
vectorizer = CountVectorizer(vocabulary=vocab_dict)
X = vectorizer.fit_transform(corpus)
X.toarray()

------------------------------------------

Getting File paths to raw data

In [1]:
import scipy.sparse
import numpy as np
import os 
import enchant
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
from sklearn import metrics
from nltk.corpus import stopwords
from scipy.spatial import distance
from scipy import sparse
from sklearn.preprocessing import normalize
from datetime import datetime

# HELPPER FUNCTIONS

# Given a directory path, get all of the file names and join the path to create an absolute path
def GetListOfFileNames(dir_path):
    return [os.path.join(dir_path, f) for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]


# Calculate a the projection of vector a onto b, where both a and b are scipy sprase matrix
def VectorProjection(a, b):
    numerator = np.dot(a, b)
    #print(numerator.shape)
    denominator = np.dot(b.T, b)
    #print(denominator.shape)
    frac = numerator/denominator
    return np.multiply(b, frac)


def Distance(a, b):
    #print(f"a: {a.shape}")
    #print(f"b: {b.shape}")
    diff = a - b
    sum_square = np.dot(diff, diff.T)
    #print(f"sum_square: {sum_square.shape}")
    return np.sqrt(sum_square)


# Global constant variables
CURRENT_DIRECTORY = os.getcwd()
DIR_OF_TEXT = os.path.join(CURRENT_DIRECTORY, "en_data/text")
FILE_PATH_TEXTS = GetListOfFileNames(DIR_OF_TEXT)
STOP_WORDS = stopwords.words('english')
EN_DICT = enchant.Dict("en_US")

POLAR_WORD_SETS =[
    ["negative", "positive"],
    ["displeased", "pleased"],
    ["disapproving", "approving"],
    ["disliking", "liking"],
    ["fear", "hope"],
    ["distress", "joy"],
    ["shame", "pride"],
    ["reproach", "admiration"],
    ["hate", "love"],
    ["disgust", "interest"],
    ['remorse', 'gratification'],
    ['anger', 'gratitude'],
    ["fears-confirmed", "satisfaction"],
    ["disappointment", "relief"],
    ["resentment", "happy-for"],
    ["pity", "gloating"]
]

polar_word_index = np.full((16, 2), -1, dtype=np.int_)

# Init document-token matrix
embedding_sets = {
    "count_vectorizer": text.CountVectorizer,
    "tfidf_vectorizer": text.TfidfVectorizer,
    "hashing_vectorizer": text.HashingVectorizer
}


CURRENT_SET = "tfidf_vectorizer"

Processing data into embedding space

In [2]:
# Hour long computation, only do if necessary...

print(f"Finding all valid tokens in Documents...")
vectorizer_spgc = embedding_sets[CURRENT_SET](  input="filename", 
                                    encoding='utf-8',
                                    decode_error='ignore',
                                    strip_accents='unicode',
                                    lowercase=True,
                                    stop_words=STOP_WORDS,
                                    ngram_range = (1, 1),
                                    analyzer='word',
                                    dtype=np.float32)

# Construct document-token matrix, then transpose for token-document matrix
count_embeding = vectorizer_spgc.fit_transform(FILE_PATH_TEXTS)

print(f"term_freq_document shape: {count_embeding.transpose().shape}")
feature_list = vectorizer_spgc.get_feature_names_out()

print(f"Finding valid words from tokenizer...")
sub_word_list = list()
for token in feature_list:
    if any(char.isdigit() for char in token):
        continue
    if EN_DICT.check(token):
        sub_word_list.append(token)

fraction_of_words = len(sub_word_list)/len(feature_list)

print(f"From the {len(feature_list)} tokens, only {len(sub_word_list)} are words. A {(1.0 - fraction_of_words) * 100.0}% reduction")

vocab_dict = {word.lower(): idx for idx, word in enumerate(sub_word_list)}

print(f"Using words as vocabulary for embedding step...")
vectorizer_spgc = embedding_sets[CURRENT_SET](  input="filename", 
                                                encoding='utf-8',
                                                decode_error='ignore',
                                                strip_accents='unicode',
                                                lowercase=True,
                                                stop_words=STOP_WORDS,
                                                ngram_range = (1, 1),
                                                analyzer='word',
                                                vocabulary= vocab_dict,
                                                dtype=np.float32)

# Construct document-token matrix, then transpose for token-document matrix
count_embeding = vectorizer_spgc.fit_transform(FILE_PATH_TEXTS)
count_embeding = count_embeding.transpose()

print(f"term_freq_document shape: {count_embeding.shape}")
feature_list = vectorizer_spgc.get_feature_names_out()

Finding all valid tokens in Documents...
term_freq_document shape: (6077522, 19369)
Finding valid words from tokenizer...
From the 6077522 tokens, only 100024 are words. A 98.35419764831785% reduction
Using words as vocabulary for embedding step...
term_freq_document shape: (100024, 19369)


In [3]:
FILE_PATH_TEXTS.clear()

In [4]:
# From the vocabulary created by CountVectorizer, find the token index. If not found, keep -1
for word_set_idx in range(len(POLAR_WORD_SETS)):
    for word_idx in range(len(POLAR_WORD_SETS[word_set_idx])):
        for (index, item) in enumerate(feature_list):
            if item == POLAR_WORD_SETS[word_set_idx][word_idx]:
                print(f"{item} at index {index}")
                polar_word_index[word_set_idx][word_idx] = index
                break
print(polar_word_index)

negative at index 58054
positive at index 67101
displeased at index 25911
pleased at index 66092
disapproving at index 25212
approving at index 4291
disliking at index 25752
liking at index 51006
fear at index 32852
hope at index 42358
distress at index 26239
joy at index 48296
shame at index 79265
pride at index 68285
reproach at index 73935
admiration at index 1198
hate at index 40568
love at index 51970
disgust at index 25626
interest at index 46596
remorse at index 73527
gratification at index 38806
anger at index 3202
gratitude at index 38820
satisfaction at index 77039
disappointment at index 25204
relief at index 73320
resentment at index 74150
pity at index 65752
gloating at index 37926
[[58054 67101]
 [25911 66092]
 [25212  4291]
 [25752 51006]
 [32852 42358]
 [26239 48296]
 [79265 68285]
 [73935  1198]
 [40568 51970]
 [25626 46596]
 [73527 38806]
 [ 3202 38820]
 [   -1 77039]
 [25204 73320]
 [74150    -1]
 [65752 37926]]


count_embeding is a document-term matrix, where each row is a document and each featrue is a token. We may want to transpose this so that the first index is the word, and the second index is the document.

From here, we want to perform vector arithmatic to determine how "similar" the words are in relation to a polar set of words. To do this we will need to:

    - Get the polar word vectors (hate/love)
    - Determine the vector between these by subtracting each other (love - hate)
    - SemAxis will then need to be normalized ([love-hate]/|[love-hate]|)
    - Project all terms vectors onto this SemAxis
    - Determine the distance of the projected term from the negative word
    - normalize distance such that it is from [0-1]

![image.png](pic/revised_occ.png)

From the above list, we can see which words are in the embeddings space and which are not. Suspicion confrimed that the emotional words "fears-confrimed" and "happy-for" is not in the embedding. Will need to find a replacement word.

To calculate the valence of the word, we will perform the following steps.

- Transpose the document-term matrix to term-document. 
- Get the negative and positive tokens, subtrack their vectors to generate a SemAxis and normalize the vector. Below is the semaxis for the polar words (Negative, Positive)
- Project all token vectors onto the SemAxis vector.

To save time, we will use scikit learn's distance matrix function to calculate the distance between each word and midpoint polar set

In [5]:
polar_set_midpoints = np.zeros(shape=(len(POLAR_WORD_SETS), count_embeding.shape[1]), dtype=np.float32)
semaxis_norms = np.zeros(shape=(len(POLAR_WORD_SETS), count_embeding.shape[1]), dtype=np.float32)
polar_set_strings = list()


for word_set_idx in range(len(POLAR_WORD_SETS)):
    negative_idx = polar_word_index[word_set_idx][0] 
    positive_idx = polar_word_index[word_set_idx][1]
    negative_str = POLAR_WORD_SETS[word_set_idx][0]
    positive_str = POLAR_WORD_SETS[word_set_idx][1]
    
    if negative_idx == -1 or positive_idx == -1:
        continue 
    
    negative_point = count_embeding[negative_idx].toarray()
    positive_point = count_embeding[positive_idx].toarray()
    pos_neg_vector = positive_point - negative_point
    semaxis_norm = pos_neg_vector / np.sqrt(np.sum(pos_neg_vector**2))
    midpoint = pos_neg_vector/2.0

    semaxis_norms[word_set_idx] = semaxis_norm
    polar_set_midpoints[word_set_idx] = midpoint[0]
    polar_set_strings.append(f"{negative_str}_{positive_str}")

polar_set_midpoints = sparse.csr_matrix(polar_set_midpoints, dtype=np.float32)
semaxis_norms = sparse.csr_matrix(semaxis_norms, dtype=np.float32)

print(polar_set_midpoints.shape)
print(semaxis_norms.shape)

(16, 19369)
(16, 19369)


In [6]:
print(f"Normalizing count embeding...")
count_embeding_normalized = normalize(count_embeding, norm='l2', axis=1)

## Calculate projected word position
print(f"Calculating projected word...")
print(f"...numerator calculation...")
numerator = metrics.pairwise.cosine_similarity( X = count_embeding_normalized,
                                                Y = semaxis_norms)
numerator = sparse.csr_matrix(numerator)
scalar = sparse.linalg.norm(x=count_embeding.transpose(), axis=0)
scalar = np.resize(a=scalar, new_shape=(count_embeding.shape[0], len(POLAR_WORD_SETS)))
print(f"numerator: {numerator.shape}")
print(f"scalar: {scalar.shape}")

print(f"...denominator calculation...")
denominator = sparse.linalg.norm(   x=semaxis_norms,
                                    axis=1)

#denominator = np.linalg.norm(x=semaxis_norms, axis=1)
#denominator = np.resize(a=denominator, new_shape=(semaxis_norms.shape[1], len(POLAR_WORD_SETS)))

numerator = numerator.multiply(scalar)
print(f"denominator: {denominator.shape}")

print(f"...projection scalar calculation...")
#word_frac = np.divide(numerator, denominator)
word_frac = numerator/denominator
word_frac = sparse.csr_matrix(word_frac)
print(f"word_frac: {word_frac.shape}")



Normalizing count embeding...
Calculating projected word...
...numerator calculation...
numerator: (100024, 16)
scalar: (100024, 16)
...denominator calculation...
denominator: (16,)
...projection scalar calculation...
word_frac: (100024, 16)


C:\Users\Jake Klinkert\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\base.py:595: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


In [9]:
# This block is questionable, sometimes it asks for too much memory, and othertimes it works fine.

print(f"...final word projection calculation...")
dir_for_set = os.path.join(CURRENT_DIRECTORY, f"en_data/{CURRENT_SET}/")
if not os.path.isdir(dir_for_set):
    os.mkdir(dir_for_set)

for polar_set_idx in range(len(POLAR_WORD_SETS)):
    negative_idx = polar_word_index[polar_set_idx][0] 
    positive_idx = polar_word_index[polar_set_idx][1]
    negative_str = POLAR_WORD_SETS[polar_set_idx][0]
    positive_str = POLAR_WORD_SETS[polar_set_idx][1]

    if negative_idx == -1 or positive_idx == -1:
        continue

    print(f"...Calculating polar set {polar_set_idx}_{negative_str}_{positive_str}...")
    #print(f"semaxis_norms: {semaxis_norms[polar_set_idx].shape}")
    #print(f"word_frac: {word_frac.T[polar_set_idx].shape}")
    word_projected_point = semaxis_norms[polar_set_idx].T @ word_frac.T[polar_set_idx]
    word_projected_point = sparse.csr_matrix(word_projected_point.T)
    #print(f"word_projected_point: {word_projected_point.shape}")

    # Distance of the word from the midpoint
    word_distance = metrics.pairwise.pairwise_distances(X=word_projected_point,
                                                        Y=midpoint,
                                                        metric = "euclidean",
                                                        n_jobs=-1,
                                                        force_all_finite = True)

    #print(f"count_embeding_normalized: {count_embeding_normalized[1]}")
    #print(f"word_projected_point: {word_projected_point[0]}")
    #print(f"word_distance: {word_distance}")
    max = np.amax(word_distance) 
    word_distance = word_distance/max
    word_distance = sparse.csr_matrix(word_distance)
    # Used to determine polarity
    word_dot_product = metrics.pairwise.linear_kernel(  X = count_embeding_normalized,
                                                        Y = semaxis_norms)

    dir_for_polar = os.path.join(dir_for_set, f"{polar_set_idx}_{negative_str}_{positive_str}.csv")


    #print(f"word_distance:{word_distance.shape}")
    #print(f"word_dot_product:{word_dot_product.T[polar_set_idx].shape}")

    with open(dir_for_polar, 'w') as textfile:
        textfile.write("Word, Value\n")
        for idx in range(word_distance.shape[0]):
            sign = 1.0
            if word_dot_product.T[polar_set_idx].T[idx] < 0.0:
                sign = -1.0
            textfile.write(f"{feature_list[idx]},{(word_distance[idx].todense() * sign)}\n")

    del word_projected_point
    del word_distance
    del word_dot_product

...final word projection calculation...
...Calculating polar set 0_negative_positive...
...Calculating polar set 1_displeased_pleased...
...Calculating polar set 2_disapproving_approving...
...Calculating polar set 3_disliking_liking...
...Calculating polar set 4_fear_hope...
...Calculating polar set 5_distress_joy...
...Calculating polar set 6_shame_pride...
...Calculating polar set 7_reproach_admiration...
...Calculating polar set 8_hate_love...
...Calculating polar set 9_disgust_interest...
...Calculating polar set 10_remorse_gratification...
...Calculating polar set 11_anger_gratitude...
...Calculating polar set 13_disappointment_relief...
...Calculating polar set 15_pity_gloating...
